In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow


!wget -O MobileNetSSD_deploy.prototxt https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt
!wget -O MobileNetSSD_deploy.caffemodel https://github.com/chuanqi305/MobileNet-SSD/raw/master/mobilenet_iter_73000.caffemodel


model = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt', 'MobileNetSSD_deploy.caffemodel')


CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow",
           "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

VEHICLE_CLASSES = ["aeroplane", "bicycle", "boat", "bus", "car", "motorbike", "train"]

from google.colab import files
uploaded = files.upload()

video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter('output_video_with_signal.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))


frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break


    if frame_count % 2 == 0:
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)


        model.setInput(blob)
        detections = model.forward()


        person_count = 0
        vehicle_count = 0


        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            if confidence > 0.3:
                idx = int(detections[0, 0, i, 1])
                label = CLASSES[idx]


                if label == "person":
                    person_count += 1
                elif label in VEHICLE_CLASSES:
                    vehicle_count += 1


                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, f"{label}: {confidence * 100:.2f}%", (startX, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


        if person_count > vehicle_count:
            cv2.circle(frame, (50, 50), 20, (0, 255, 0), -1)
            cv2.putText(frame, "More persons detected, all clear!", (80, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        elif vehicle_count > person_count:
            cv2.circle(frame, (50, 50), 20, (0, 0, 255), -1)
            cv2.putText(frame, "Vehicles identified, stay safe!", (80, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            cv2.circle(frame, (50, 50), 20, (0, 255, 255), -1)
            cv2.putText(frame, "Equal persons and vehicles, caution!", (80, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)


    output.write(frame)
    frame_count += 1


cap.release()
output.release()


from google.colab import files
files.download('output_video_with_signal.mp4')


--2024-09-26 06:32:11--  https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44667 (44K) [text/plain]
Saving to: ‘MobileNetSSD_deploy.prototxt’

MobileNetSSD_deploy 100%[===================>]  43.62K  --.-KB/s    in 0.01s   

2024-09-26 06:32:12 (3.57 MB/s) - ‘MobileNetSSD_deploy.prototxt’ saved [44667/44667]

--2024-09-26 06:32:12--  https://github.com/chuanqi305/MobileNet-SSD/raw/master/mobilenet_iter_73000.caffemodel
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/mobilenet_iter_73000